In [ ]:
#setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA
#visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


In [ ]:
#load data
df = pd.read_csv('Coffee Chain.csv')

#display basic information
print("Dataset Shape:", df.shape)
print("\nFirst few rows:")
df.head()

In [ ]:
#data exploration
print("Data Types:")
print(df.dtypes)
print("\n" + "="*50)
print("\nMissing Values:")
print(df.isnull().sum())
print("\nBasic Statistics:")
df.describe()


In [ ]:
#clean column names, remove extra quotes
for col in df.columns:
    df[col] = df[col].apply(lambda x: x.strip('"""') if isinstance(x, str) else x)

#convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'])

#check unique values for categorical variables
print("Market Types:", df['Market'].unique())
print("\nMarket Sizes:", df['Market Size'].unique())
print("\nProduct Lines:", df['Product Line'].unique())
print("\nProduct Types:", df['Product Type'].unique())
print("\nCaffeine Types:", df['Type'].unique())


In [ ]:
#select features for clustering
features_for_clustering = ['Sales', 'Profit', 'Margin', 'Marketing', 'Inventory', 
                          'Difference Between Actual and Target Profit']

#create feature matrix
X = df[features_for_clustering].copy()

#display feature statistics
print("Feature Statistics:")
print(X.describe())
print("\n" + "="*50)
print("\nFeature Correlations:")
correlation_matrix = X.corr()
print(correlation_matrix)


In [ ]:
#visualize feature correlations
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#convert back to DataFrame for easier handling
X_scaled_df = pd.DataFrame(X_scaled, columns=features_for_clustering)

print("Standardized Features - First 10 rows:")
print(X_scaled_df.head(10))
print("\n" + "="*50)
print("\nStandardized Features Statistics (should be mean≈0, std≈1):")
print(X_scaled_df.describe())

In [ ]:
#determine optimal number of clusters
k_range = range(2, 11)
wcss = []
silhouette_scores = []
davies_bouldin_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))
    davies_bouldin_scores.append(davies_bouldin_score(X_scaled, kmeans.labels_))

# Create results DataFrame
results_df = pd.DataFrame({
    'k': list(k_range),
    'WCSS': wcss,
    'Silhouette Score': silhouette_scores,
    'Davies-Bouldin Index': davies_bouldin_scores
})

print(results_df)

In [ ]:
#elbow plot
axes[0].plot(k_range, wcss, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Within-Cluster Sum of Squares', fontsize=12)
axes[0].set_title('Elbow Method', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

#silhouette score plot
axes[1].plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score (Higher is Better)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

#davies-Bouldin Index plot
axes[2].plot(k_range, davies_bouldin_scores, 'ro-', linewidth=2, markersize=8)
axes[2].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[2].set_ylabel('Davies-Bouldin Index', fontsize=12)
axes[2].set_title('Davies-Bouldin Index (Lower is Better)', fontsize=14, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#find optimal k
optimal_k_silhouette = k_range[np.argmax(silhouette_scores)]
optimal_k_db = k_range[np.argmin(davies_bouldin_scores)]

print(f"\nOptimal k by Silhouette Score: {optimal_k_silhouette}")
print(f"Optimal k by Davies-Bouldin Index: {optimal_k_db}")
print(f"\nSelected k: 4 (based on elbow method showing diminishing returns and good silhouette score)")


In [ ]:
#perform final clustering with optimal k
optimal_k = 4
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['Cluster'] = kmeans_final.fit_predict(X_scaled)

#add cluster labels to the original dataframe
print(f"Clustering completed with k={optimal_k}")
print(f"\nCluster distribution:")
print(df['Cluster'].value_counts().sort_index())
print(f"\nPercentage distribution:")
print(df['Cluster'].value_counts(normalize=True).sort_index() * 100)


In [ ]:
#calculate cluster centers (in original scale)
cluster_centers = df.groupby('Cluster')[features_for_clustering].mean()
print("Cluster Centers (Average Values):")
print(cluster_centers.round(2))

In [ ]:
#analyze cluster characteristics including categorical variables
for cluster_id in range(optimal_k):
    print(f"\n{'='*80}")
    print(f"CLUSTER {cluster_id} CHARACTERISTICS")
    
    cluster_data = df[df['Cluster'] == cluster_id]
    
    #numerical statistics
    print(f"\nSize: {len(cluster_data)} records ({len(cluster_data)/len(df)*100:.1f}%)")
    print("\nNumerical Features (mean):")
    print(cluster_data[features_for_clustering].mean().round(2))
    
    #categorical distributions
    print("\nTop Product Types:")
    print(cluster_data['Product Type'].value_counts().head())
    
    print("\nTop Products:")
    print(cluster_data['Product'].value_counts().head())
    
    print("\nMarket Distribution:")
    print(cluster_data['Market'].value_counts())
    
    print("\nMarket Size Distribution:")
    print(cluster_data['Market Size'].value_counts())
    
    print("\nCaffeine Type Distribution:")
    print(cluster_data['Type'].value_counts())

In [ ]:
#PCA for 2D visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

#creating a DataFrame for plotting
pca_df = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'Cluster': df['Cluster']
})

#plot clusters in PCA space
plt.figure(figsize=(12, 8))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['Cluster'], 
                     cmap='viridis', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]:.2%} variance)', fontsize=12)
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]:.2%} variance)', fontsize=12)
plt.title('Cluster Visualization in PCA Space', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Total variance explained by 2 components: {pca.explained_variance_ratio_.sum():.2%}")


In [ ]:
#heatmap of standardized cluster centers
cluster_centers_scaled = pd.DataFrame(
    kmeans_final.cluster_centers_,
    columns=features_for_clustering,
    index=[f'Cluster {i}' for i in range(optimal_k)]
)

plt.figure(figsize=(12, 6))
sns.heatmap(cluster_centers_scaled, annot=True, fmt='.2f', cmap='RdYlGn', 
            center=0, linewidths=1, cbar_kws={'label': 'Standardized Value'})
plt.title('Cluster Centers (Standardized Features)', fontsize=14, fontweight='bold')
plt.xlabel('Features', fontsize=12)
plt.ylabel('Clusters', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:

summary_table = df.groupby('Cluster').agg({
    'Sales': ['mean', 'std', 'min', 'max'],
    'Profit': ['mean', 'std', 'min', 'max'],
    'Margin': ['mean', 'std'],
    'Marketing': ['mean', 'std'],
    'Inventory': ['mean', 'std'],
    'Difference Between Actual and Target Profit': ['mean', 'std']
}).round(2)
print(summary_table)